<a href="https://colab.research.google.com/github/paryoja/deeplearningstudy/blob/master/Tensorflow_Basics_(2)_Variables_and_Placeholders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 변수

프로그래밍을 하기위해서는 상수뿐만이 아니라 변수도 필요합니다.

변수는 Variable 클래스의 인스턴스를 생성함으로써 만들 수 있습니다.

https://www.tensorflow.org/api_docs/python/tf/Variable 에서 생성자를 볼 수 있습니다.

```
__init__(
    initial_value=None,
    trainable=True,
    collections=None,
    validate_shape=True,
    caching_device=None,
    name=None,
    variable_def=None,
    dtype=None,
    expected_shape=None,
    import_scope=None,
    constraint=None,
    use_resource=None,
    synchronization=tf.VariableSynchronization.AUTO,
    aggregation=tf.VariableAggregation.NONE
)
```

파이썬의 class는 일반적으로 첫글자를 대문자로 이름을 붙이며 (함수는 소문자)

\_\_init\_\_() 함수는 class의 생성자입니다.

첫번째 인자는 변수의 초기값이라는 것을 기억해두세요!


In [0]:
import tensorflow as tf

x = tf.Variable([3, 4])

print(x)

variable도 sess을 거치지 않으면 단지 어떻게 값을 저장하고 있을 것인지
-초기화는 어떤식으로 할것이고, dtype은 뭐고 생김새는 어떻고-
만을 정의해둔 것입니다.

따라서 그 값을 보고 싶으면 sess.run 함수에 넣어주어야 합니다.

In [0]:
sess = tf.Session()

sess.run(x)

하지만 상수와는 다르게 뭔가 엄청 긴 에러가 발생합니다.

이럴땐 겁먹지 말고, 맨 마지막을 보시면 됩니다.

텐서플로우의 에러메시지는 이처럼 매우 길지만, 대부분의 경우 맨 마지막만 보시면 원인을 파악하실수 있습니다.


```
FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[{{node _retval_Variable_0_0}} = _Retval[T=DT_INT32, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable)]]
```

초기화되지 않은 변수를 사용하려고 했다라고 되어 있습니다.

앞에서 변수도 어떻게 값을 들고 있을 건지만을 정의되어 있지 실제로 초기화는 이루어 지지 않은 상황이기 때문에 변수의 값을 접근하려는데 문제가 발생한 것입니다.

변수를 초기화를 해주는 방법은

sess.run 함수에

tf.global_variables_initializer() 를 실행시켜주는 것입니다.


In [0]:
sess.run(tf.global_variables_initializer())

print(sess.run(x))

변수와 상수가 다른 점은 변수에는 값을 할당 할 수 있다는 것입니다.

값의 할당은 tf.assign을 통해서 수행할 수 있습니다.

In [0]:
assign = tf.assign(x, x + tf.constant(1))

x + tf.constant(1) 을 수행 한 결과는 무엇일까요?

broadcasting이 일어남에 주의해서 따져봅시다.

그렇다면 

x의 값은 그 값으로 변해 있을까요?

In [0]:
print(sess.run(x))

값이 변하지 않습니다. 초기화한 값 그대로 있습니다.

왜냐면, tf.assign은 이렇게 assign을 하겠다 라고 정의한 것이지 실제 assign을 수행하는것이 아니기 때문입니다.

생각해봅시다. x + tf.constant(1) 값은 sess만이 구해줄 수 있습니다. 

tf.assign(x, x + tf.constant(1)) 이 수행되는 시점에서는 그 값이 어느 값인지 모르는 상황이기 때문에, assign을 하고 싶어도 sess 없이는 할 수 없습니다.

따라서 tf.assign도 sess에게 넘겨줘서 명시적으로 수행시켜야 합니다.


이러한 점이 헷갈린다면, tensorflow은 매우 게을러서 명시적으로 하라고 하기 전에는 하지 않는다!

라는 것을 기억해두면 좋습니다.

In [0]:
sess.run(assign)

print(sess.run(x))

이렇게 어떻게 연산할 것인지를 **정의** 해두는 것과

**실행** 하는 것이 구분되어 있음에 유의하세요.

이렇게 구분이 되어 있음으로써

다음과 같이 한번 정의한걸 반복해서 사용할 수도 있습니다.

In [0]:
sess.run(assign)

print(sess.run(x))

sess.run(assign)

print(sess.run(x))

sess.run(assign)

print(sess.run(x))

sess.run(assign)

print(sess.run(x))

# Placeholder

주어진 값 x에 5을 더하는 연산이 있다고 해봅시다.

x에 1을 넣으면 6을 계산하고, 5를 넣으면 10를 계산해줄텐데요.

만약 이런 입력을 그떄그때 받을 수 있도록 하는 경우에

Variable이나 constant를 쓴다면,

매번 변수를 만들고, 초기화하고 신경을 써줘야할 것이 많습니다.

또한 변수나 상수로 만들면 그만큼 메모리 사용량이 늘기 때문에 좋지 못합니다.


이 경우 텐서플로우에서는 placeholder라는 Tensor를 만들어 사용할 수 있습니다.


placeholder는 변수나 상수와 같이 사용할 수 있습니다.


In [0]:
import tensorflow as tf

variable_x = tf.Variable(5.0, dtype=tf.float32)

placeholder_y = tf.placeholder(dtype=tf.float32)

x_plus_y = variable_x + placeholder_y

위 코드는 5.0으로 초기화 될 variable_x 와 float32 형 값을 받을 placeholder를 선언하고 이를 더하는 연산을 만들었습니다.

다시 반복하지만, 이를 연산하기 위해서는 sess이 필요합니다.

Variable을 쓰고 있으므로 tf.global_variables_initializer()를 호출하는 것을 잊지 마세요.

In [0]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

sess.run(x_plus_y)

(의도한 바이지만) 역시나 에러가 나네요.

variable_x의 값은 초기화가 되어 있어 5.0 이지만,

placeholder_y의 값은 무엇인가요? 위의 코드로는 전혀 알 수 없습니다.

따라서 placeholder가 포함된 연산을 수행 시키기 위해서는

해당 placeholder에 들어갈 값이 무엇인지를 명시해줘야 합니다.

이는 feed_dict 라는 argument에 dictionary 형태로 값을 적어 주면 됩니다.



In [0]:
print(sess.run(x_plus_y, feed_dict={placeholder_y: 4.0}))
      
     

```
{placeholder_y: 4.0} 
```
위 dictionary를 통해 라이브러리는 placeholder_y의 위치에 어떠한 값을 써야하는지 비로소 알 수 있습니다.

Placeholder y의 값으로 4.0을 넣어줬더니 4.0 + 5.0 이 계산되서 9.0이 출력 된 것을 보실 수 있습니다.


## Variable 사용시 주의점
sess.run(tf.global_variables_initializer()) 호출을 잊지 말자


## Placeholder 사용시 주의점
sess.run() 실행시 feed_dict를 넣어주자


그럼 위 코드를 재사용해서 5.0 + 15.0 을 계산해 볼까요?

In [0]:
print(sess.run(x_plus_y, feed_dict={placeholder_y: 15.0}))

새로운 노드를 생성하지 않고 덧셈을 수행할 수 있었습니다.

나중에는 tf.dataset이라는 라이브러리로 데이터를 넣어주겠지만,

당분간은 placeholder를 이용하여 데이터를 넣어주겠습니다.

## Placeholder의 shape

placeholder 라이브러리를 보겠습니다.

```
tf.placeholder(
    dtype,
    shape=None,
    name=None
)
```

dtype은 반드시 필요하고,

shape과 name은 optional 입니다.

name은 앞에서도 몇번 봤고,

shape은 지정하지 않으면 어떠한 shape도 placeholder로 전달해줄 수 있습니다.

예를 들면, 위에서 정의한 x_plus_y를 재사용할수 있습니다.

y에 \[\[1, 2\], \[3, 4\]\] 와 같은 matrix를 전달해 보도록 하겠습니다.

In [0]:
print(sess.run(x_plus_y, feed_dict={placeholder_y: [[1, 2],[3, 4]] }))

x는 스칼라이므로 broadcasting이 발생하여 각 element에 5씩 더해준 결과값을 얻었습니다.

물론 이렇게 구성하는게 필요한 경우도 있겠지만,

대부분의 경우 입력으로 주는 값의 shape은 고정되어 있기 때문에 

placeholder 생성시부터 shape을 명시해주는것이 버그를 줄이는 지름길입니다.


In [0]:
placeholder_z = tf.placeholder(tf.float32, shape=[2, 2])

x_plus_z = variable_x + placeholder_z

print(sess.run(x_plus_z, feed_dict={placeholder_z: [[1, 2],[3, 4]]}))

다른 shape을 입력으로 주려는 경우 다음과 같은 에러가 발생합니다.

In [0]:
print(sess.run(x_plus_z, feed_dict={placeholder_z: [1, 2]}))

```
ValueError: Cannot feed value of shape (2,) for Tensor 'Placeholder_4:0', which has shape '(2, 2)'
```

2짜리 벡터를 2,2 로 선언한 placeholder에 넘겨줄수 없다는 것입니다.

다양한 에러를 만들어 보시면 실제 구현중에 에러가 났을때 원인을 쉽게 규명할수 있습니다.



---


때로는 \[2, 2\] 행렬을 feed 하고 싶고, 때로는 \[3, 2\] 행렬을 feed하고 싶을 수도 있습니다.

이 경우에는 shape를 명시할때, 바뀔수 있는 숫자의 위치에 None을 써주시면 됩니다.

In [0]:
placeholder_w = tf.placeholder(tf.float32, shape=[None, 2])

x_plus_w = variable_x + placeholder_w

print(sess.run(x_plus_w, feed_dict={placeholder_w: [[1, 2],[3, 4]]}))
print()
print(sess.run(x_plus_w, feed_dict={placeholder_w: [[1, 2], [3, 4], [5, 6]]}))